In [1]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.1 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-4e6mvyvo
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-4e6mvyvo
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369410 sha256=01310c825d057c3e7e9494e7293c77fbbe0c67503db4748808bd19b4d55b4fee
  Stored in directory: /tmp/pip-ephem-wheel-cache-csqat0o1/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [2]:
import random
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import cv2
import torch.nn as nn
import numpy as np
import torch
import os
import clip
from PIL import Image
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from pkg_resources import packaging
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn import metrics
from torch.autograd import Variable
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [3]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14@336px", device=device)

100%|███████████████████████████████████████| 891M/891M [00:54<00:00, 17.3MiB/s]


In [5]:
train_path= "/kaggle/input/vizwiz/Annotations/Annotations/train.json"
val_path= "/kaggle/input/vizwiz/Annotations/Annotations/val.json"
test_path = "/kaggle/input/vizwiz/Annotations/Annotations/test.json"

with open(train_path) as f:
    train_data = json.load(f)
with open(val_path) as f:
    val_data = json.load(f)
with open(test_path) as f:
    test_data = json.load(f)

path_train_images="/kaggle/input/vizwiz/train/train"
path_val_images="/kaggle/input/vizwiz/val/val"
path_test_images="/kaggle/input/vizwiz/test/test"

train=pd.DataFrame(train_data)
val=pd.DataFrame(val_data)
test=pd.DataFrame(test_data)

print(train.shape)
print(val.shape)
print(test.shape)

(20523, 5)
(4319, 5)
(8000, 2)


In [6]:
random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(train.iloc[:,[0,1,2,4]],train.iloc[:,3], test_size=0.05, random_state=42)
print(X_train.shape)
print(X_test.shape)

(19496, 4)
(1027, 4)


In [10]:
data1=np.array(X_train.iloc[:][['image','question']])
data2=torch.empty((data1.shape[0],1536), dtype=torch.float).cuda()
print(data1.shape)
for i in range(len(data1)):
    with torch.no_grad():
            text_features=model.encode_text(clip.tokenize(data1[i,1]).cuda()).cuda()
            image = Image.open(path_train_images+'/'+data1[i,0])
            image = preprocess(image).unsqueeze(0).cuda()
            image_features=model.encode_image(image).cuda()
            combined_features=torch.cat([image_features,text_features], dim=-1).cuda()
    data2[i]=combined_features.cuda()
print(data2.shape)
torch.save(data2, "/kaggle/working/combined_features.pt")

(19496, 2)
torch.Size([19496, 1536])


In [11]:
data1=np.array(np.array(val.iloc[:][['image','question']]))
data2=torch.empty((data1.shape[0],1536), dtype=torch.float).cuda()
print(data1.shape)
for i in range(len(data1)):
    with torch.no_grad():
            text_features=model.encode_text(clip.tokenize(data1[i,1]).cuda()).cuda()
            image = Image.open(path_val_images+'/'+data1[i,0])
            image = preprocess(image).unsqueeze(0).cuda()
            image_features=model.encode_image(image).cuda()
            combined_features=torch.cat([image_features,text_features], dim=-1).cuda()
    data2[i]=combined_features.cuda()
print(data2.shape)
torch.save(data2, "/kaggle/working/val_combined_features.pt")

(4319, 2)
torch.Size([4319, 1536])


In [13]:
data1=np.array(np.array(X_test.iloc[:][['image','question']]))
data2=torch.empty((data1.shape[0],1536), dtype=torch.float).cuda()
print(data1.shape)
for i in range(len(data1)):
    with torch.no_grad():
            text_features=model.encode_text(clip.tokenize(data1[i,1]).cuda()).cuda()
            image = Image.open(path_train_images+'/'+data1[i,0])
            image = preprocess(image).unsqueeze(0).cuda()
            image_features=model.encode_image(image).cuda()
            combined_features=torch.cat([image_features,text_features], dim=-1).cuda()
    data2[i]=combined_features.cuda()
print(data2.shape)
torch.save(data2, "/kaggle/working/test_combined_features.pt")

(1027, 2)
torch.Size([1027, 1536])
